In [1]:
import pandas as pd

import psycopg2
host = '/var/run/postgresql'
sqluser = 'sudarshan'
dbname = 'mimic'

from QuickUMLS.quickumls import QuickUMLS
umlspath = '/storage/UMLS-Storage/quickumls'

In [2]:
def total_len(l):
    return sum([len(sl) for sl in l])

In [ ]:
cats = pd.read_csv('cats.csv')
max_limit = 5

queries = []
for category, n_notes in zip(cats['category'], cats['number_of_notes']):
    limit = min(max_limit, n_notes) if max_limit > 0 else n_notes
    if limit == max_limit:
        q = f"""
        select category, text from correctnotes where category=\'{category}\' order by random() limit {limit};
        """
    else:
        q = f"""
        select category, text from correctnotes where category=\'{category}\';
        """
    queries.append(q)

dfs = []

con = psycopg2.connect(dbname=dbname, user=sqluser, host=host)
for q in queries:
    df = pd.read_sql_query(q, con)
    dfs.append(df)
con.close()
    
df = pd.concat(dfs)
# df.set_index('row_id', inplace=True)
df.shape

In [3]:
matcher = QuickUMLS(umlspath, threshold=0.9, similarity_name='cosine')

In [9]:
from QuickUMLS import toolbox

In [14]:
cui_vecs = pd.read_csv('/storage/pre-trained/cui2vec_pretrained.csv')
cui_vecs.set_index('cui', inplace=True)
cui_vecs.shape

(109053, 500)

In [11]:
test_note = "Patient recently says she has brown vaginal discharge. She also shows signs of cardiac arrest and has type 2 diabetes."

In [12]:
matches = matcher.match(test_note, best_match=True, ignore_syntax=False)
l = len(matches)
tl = total_len(matches)
al = tl/l
l, tl, al

(4, 6, 1.5)

In [19]:
cuis = []
terms = {}
for cs in matches:
    for c in cs:
        if c['term'] not in terms:
            terms[c['term']] = True
            cuis.append(c['cui'])

In [21]:
cui_vecs.sample()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V491,V492,V493,V494,V495,V496,V497,V498,V499,V500
cui,,,,,,,,,,,,,,,,,,,,,
C2849540,-0.142322,-0.121415,1.110223e-16,-0.008351,0.010817,0.000942,0.00359,-0.012414,-0.025052,-2.289835e-16,...,0.07434,0.018911,-0.066674,0.143624,-0.084779,-0.101644,-0.062375,-0.126647,-0.097057,-0.031699


In [32]:
matches[3]

[{'start': 0,
  'end': 7,
  'ngram': 'Patient',
  'term': 'Patient',
  'cui': 'C1550655',
  'similarity': 1.0,
  'semtypes': {'T031'},
  'preferred': 1},
 {'start': 0,
  'end': 7,
  'ngram': 'Patient',
  'term': 'Patient',
  'cui': 'C1578485',
  'similarity': 1.0,
  'semtypes': {'T170'},
  'preferred': 1},
 {'start': 0,
  'end': 7,
  'ngram': 'Patient',
  'term': 'Patient',
  'cui': 'C1578486',
  'similarity': 1.0,
  'semtypes': {'T170'},
  'preferred': 1}]

In [25]:
cuis

['C0241623', 'C0011860', 'C0018790', 'C1550655']

In [29]:
cui_vecs.loc[cuis[3]].values

KeyError: 'the label [C1550655] is not in the [index]'

In [ ]:
test_note = '''
Joseph R. Smith

1234567-8

4/5/2006

REASON FOR VISIT:Here to get a new primary care physician.

HPI: Mr. Smith is a 56-year-old gentleman formally followed at Carolina Premier who presents to obtain a new primary care physician secondary to insurance changes. He has a past medical history significant for a myocardial infarction in 1994. His cholesterol has been fine. Catheterization showed a possible "kink" in one of his vessels and it was thought that he had a possible "eddy" of current which led to a clot. He has been on Coumadin since then as well as a calcium channel blocker with the thought that there may have been a superimposed spasm. He has had several unremarkable stress tests since then. He works out on a Nordic-Track three times a week without any chest pain or shortness of breath. He has a history of possible peptic ulcer disease in 1981. He was treated with H2 blockers and his symptoms resolved. He has never had any bleeding to his knowledge. He had a hernia repair bilaterally in 1989 and surgery for a right knee cyst in 1999, all of which went well. He also has about a year long history of right buttock pain. This happens only when he is sitting for some time and does not change position. It does not happen when he is walking or exercising. He wonders if it might be pyriformis syndrome. If he changes positions frequently or stretches his legs, this seems to help. He has no acute complaints today and is here to get plugged into the system. He does wonder if there is anything else that can be done about his buttock pain.

PAST MEDICAL HISTORY: As above. In addition, he had a flexible sigmoidoscopy in 2003 which was okay.

MEDICATIONS: Tylenol p.r.n., baby aspirin p.o. q.d., Coumadin 3.5 mg p.o. q.d., Adalat Time Released tablets 30 mg one p.o. q.d., multivitamin, glucosamine and chondroitin sulfate.

ALLERGIES: No Known Drug Allergies

FAMILY HISTORY: His mother had diabetes developed at age 55 and coronary artery disease in her mid sixties. His father had CAD as well but not until his 70s. A paternal aunt had breast cancer. There is no history of colon or prostate cancer.

SOCIAL HISTORY: He lives in Durham with his wife and mother. He works for a biotech company. He does not smoke. He drinks two beers per night and reports no trouble with alcohol in the past. No history of drug use.

REVIEW OF SYSTEMS: As per his personal health summary and is significant only for his buttock pain as listed above, but is otherwise essentially unremarkable.

PHYSICAL EXAM:

VITAL SIGNS: Weight 175.2 pounds which is 79.3 kg, blood pressure 142/96 by the nurse, 140/92 by me, pulse is 64.

GENERAL: A healthy-appearing middle-aged gentleman.

HEENT: Pupils equal, round, reactive to light. Conjunctivaepink. Sclerae anicteric. Tympanic membranes clear. Oropharynx clear.

NECK: No lymphadenopathy or thyromegaly or JVD.

LUNGS: Clear to auscultation and percussion.

HEART: Regular rate and rhythm without murmur, rub, or gallop.

ABDOMEN: Normal bowel sounds. Soft, nontender. No hepatosplenomegaly.

EXTREMITIES: No cyanosis, clubbing, or edema. 2+ peripheral pulses.

NEUROLOGIC: Motor and sensation grossly intact.

PSYCHIATRIC: Normal affect and behavior.

DERMATOLOGIC: He has a small whitish papule at the upper borderof his mustache on the left.

MUSCULOSKELETAL: Full range of motion of his legs and hips bilaterally with no tenderness to palpation over his buttocks.

ASSESSMENT/PLAN:

Status post myocardial infarction. No evidence for actual CAD. He will continue his aspirin and Coumadin. Unclear to exactly how long he should be on Coumadin or if this is really needed. At some point, may discuss this with cardiology. We will check an INR today and get him plugged into our Coumadin clinic. Check cholesterol.
High blood pressure. He will continue with diet and exercise changes. At next visit, may go up on his calcium channel blocker versus add another agent. Check creatinine and potassium today.
Buttock pain. Will try some physical therapy.
Will send him to dermatology for the papule above his mustache.
Possible h/o peptic ulcer disease. As he has not had any known GI bleed, do not feel a need to check H. pylori serology today.
Health maintenance. Tetanus shot today. Flex sig as above. Will discuss prostate cancer screening next visit.
Return to clinic in four months.
John Student, MS3

Seen with Joe Doctor, MD
'''